In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



import gradio as gr
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import gradio as gr

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Oppgaven går ut på å få maskinenlærings-modellen til å predikere hvor lenge pasienter er på sykehuset. 
#Dette basert kjønn, alder, diagnose osv.

In [ ]:
#Training data:
training_data = pd.read_csv("./training_data.csv")

td = training_data

td['gender'] = training_data['gender'].map({'F':0, 'M':1})

boolean_columns = td.select_dtypes(include=bool).columns
td[boolean_columns] = td[boolean_columns].astype(int)

td= td.drop(['vdate'], axis=1)

td['facid'] = td['facid'].map({'A':0, 'B':1, 'C':2, 'D':3, 'E':4})

td['facid'] = td['facid'].astype(float)

td['hemo'] = td['hemo'].map({True:1, False:0, 'nan':None})

td = td.drop(['id'],axis=1)

td.info()


In [7]:
#Test data:
test_data = pd.read_csv("./test_data.csv")

boolean_columns_test = test_data.select_dtypes(include=bool).columns
test_data[boolean_columns] = test_data[boolean_columns].astype(int)

test_data = test_data.drop(['id'],axis=1)

test_data

,rcount,gender,dialysisrenalendstage,asthma,irondef,pneum,substancedependence,psychologicaldisordermajor,depress,psychother,...,neutrophils,sodium,glucose,bloodureanitro,creatinine,bmi,pulse,respiration,secondarydiagnosisnonicd9,facid
0,0,0.0,0,0,0,0,0,0,0,0,...,9.550000,137.722354,141.479044,12.0,0.676542,28.702016,70,5.90,2,0.0
1,0,0.0,0,0,0,0,0,0,0,0,...,11.600000,137.861621,184.433689,12.0,1.520272,29.702723,62,6.50,1,0.0
2,4,0.0,0,1,1,0,0,0,0,0,...,16.233333,138.586277,185.799872,8.5,1.137023,29.984780,68,5.30,1,3.0
3,0,0.0,0,0,0,0,0,0,0,0,...,9.100000,142.714836,113.951944,12.0,0.845051,32.509896,84,6.50,2,1.0
4,0,0.0,0,0,0,0,0,0,0,0,...,12.000000,142.683468,111.787840,12.0,0.814372,29.870887,88,6.50,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,4,1.0,0,0,0,0,0,0,0,0,...,9.300000,136.239943,162.044287,12.0,1.018530,28.451117,77,6.50,1,0.0
29996,0,1.0,0,0,0,0,0,0,0,1,...,6.600000,139.546618,131.325169,47.0,1.158228,29.996939,71,5.85,1,4.0
29997,2,1.0,0,0,0,0,0,0,0,0,...,4.700000,136.339840,118.205945,7.0,0.861836,26.663879,47,6.50,1,0.0
29998,1,1.0,0,0,0,0,0,1,0,0,...,5.300000,136.620769,185.909652,12.0,1.038424,28.723586,91,6.80,0,4.0


In [8]:
import sklearn
from sklearn.tree import DecisionTreeRegressor


#X_train = X_train.dropna()

X_train = td.drop(columns=['lengthofstay'])
X_train = X_train.fillna(X_train.mean())
Y_train = td['lengthofstay']
X_test = test_data

model = DecisionTreeRegressor()
model.fit(X_train, Y_train)

prediction = model.predict(X_test)

prediction

array([2., 3., 7., ..., 6., 5., 1.])

In [9]:
#fjernet id fra test og training

#lager ny test data i ny variabel
test2 = pd.read_csv("./test_data.csv")

#testdata2(lengthof) = prediction
test2['lengthofstay'] = prediction

#lager ny testdata variabel -> pd.concattestdata2(id), testdata2(lengthofstay), axis=1
test3 = pd.concat((test2['id'],test2['lengthofstay']),axis=1)

#print(test3)
print(test3)

test3.to_csv("resultat.csv", index = False)

          id  lengthofstay
0      75721           2.0
1      80184           3.0
2      19864           7.0
3      76699           3.0
4      92991           4.0
...      ...           ...
29995  42648           6.0
29996  86306           3.0
29997  45466           6.0
29998  63724           5.0
29999  34122           1.0

[30000 rows x 2 columns]


In [ ]:
pd.read_csv("./data_dictionary.csv")

In [ ]:
# Function for mapping gender to int
def map_gender(gender):
    return 0 if gender == "F" else 1

def map_number(number):
    return 0 if number == "No" else 1


# Function for mapping facid to int
def map_facid(facid):
    facid_mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
    return facid_mapping.get(facid, -1)  # Default to -1 if not found


# Define a function to preprocess user inputs and save to CSV
def preprocess_input(gender, asthma,  substancedependence, psychologicaldisordermajor,
                      depress, psychother, bmi, pulse,facid,
                    #dialysisrenalendstage, #irondef, pneum,
                    #fibrosisandother, malnutrition, hemo,
                    # hematocrit, neutrophils, sodium, glucose, bloodureanitro,
                    # creatinine,
                    # respiration, secondarydiagnosisnonicd9, 
                    ):

    gender_numeric = map_gender(gender)
    facid_numeric = map_facid(facid)
    asthma_numeric = map_number(asthma)
    substancedependence_numeric = map_number(substancedependence)
    psychologicaldisordermajor_numeric = map_number(psychologicaldisordermajor)
    depress_numeric = map_number(depress)
    psychother_numeric = map_number(psychother)
    

    # Create a DataFrame from user inputs
    input_data = pd.DataFrame({
        'rcount': [0],
        'gender': [gender_numeric],
        'dialysisrenalendstage': [td['dialysisrenalendstage'].mean()],
        'asthma': [asthma_numeric],
        'irondef': [td['irondef'].mean()],
        'pneum': [td['pneum'].mean()],
        'substancedependence': [substancedependence_numeric],
        'psychologicaldisordermajor': [psychologicaldisordermajor_numeric],
        'depress': [depress_numeric],
        'psychother': [psychother_numeric],
        'fibrosisandother': [td['fibrosisandother'].mean()],
        'malnutrition': [td['malnutrition'].mean()],
        'hemo': [td['hemo'].mean()],
        'hematocrit': [td['hematocrit'].mean()],
        'neutrophils': [td['neutrophils'].mean()],
        'sodium': [td['sodium'].mean()],
        'glucose': [td['glucose'].mean()],
        'bloodureanitro': [td['bloodureanitro'].mean()],
        'creatinine': [td['creatinine'].mean()],
        'bmi': [bmi],
        'pulse': [pulse],
        'respiration': [td['respiration'].mean()],
        'secondarydiagnosisnonicd9': [td['secondarydiagnosisnonicd9'].mean()],
        'facid': [facid_numeric]
        })

    # Fill missing values in the user input data (if needed)
    input_data = input_data.fillna(input_data.mean())

    # Save user input data to a CSV file
    input_data.to_csv("user_input.csv", index=False)
    user_input = pd.read_csv("user_input.csv")

    # Make predictions using the loaded model
    prediction = model.predict(user_input)

    print(prediction)

    return prediction[0]


# Define the web interface
iface = gr.Interface(
    fn=preprocess_input,
    inputs=[gr.inputs.Radio(["F", "M"], label="Gender"),
        #    gr.inputs.Number(label="Dialysis Renal End Stage"),
        #    gr.inputs.Number(label="Asthma"),
            gr.inputs.Radio(["Yes", "No"], label="Asthma"),
        #    gr.inputs.Number(label="Iron Deficiency"),
        #    gr.inputs.Number(label="Pneumonia"),
        #    gr.inputs.Number(label="Substance Dependence"),
            gr.inputs.Radio(["Yes", "No"],label="Substance Dependence"),
        #    gr.inputs.Number(label="Psychological Disorder Major"),
            gr.inputs.Radio(["Yes", "No"],label="Psychological Disorder Major"),
        #    gr.inputs.Number(label="Depression"),
            gr.inputs.Radio(["Yes", "No"],label="Depression"),
        #    gr.inputs.Number(label="Psychotherapy"),
            gr.inputs.Radio(["Yes", "No"],label="Psychotherapy"),
        #    gr.inputs.Number(label="Fibrosis and Other"),
        #    gr.inputs.Number(label="Malnutrition"),
        #    gr.inputs.Number(label="Hemo Level"),
        #    gr.inputs.Number(label="Hematocrit"),
        #    gr.inputs.Number(label="Neutrophils"),
        #    gr.inputs.Number(label="Sodium"),
        #    gr.inputs.Number(label="Glucose"),
        #    gr.inputs.Number(label="Blood Urea Nitrogen"),
        #    gr.inputs.Number(label="Creatinine"),
            gr.inputs.Number(label="BMI"),
            gr.inputs.Number(label="Pulse"),
        #    gr.inputs.Number(label="Respiration"),
        #    gr.inputs.Number(label="Secondary Diagnosis Non-ICD9"),
            gr.inputs.Dropdown(["A", "B", "C", "D", "E"], label="Facility ID")
            ],
    outputs=gr.outputs.Textbox(label="Estimated Length of Stay")
)

iface.launch()

